In [1]:
# Create library using Mavern coordinate
# Enter "databricks:tensorframes:0.2.8-s_2.10" and search for this library
# Attach it onto your current working cluster

import tensorframes as tfs
import tensorflow as tf
from pyspark.sql import Row

In [2]:
data = [Row(x=float(x)) for x in range(10)]
df = sqlContext.createDataFrame(data)
tfs.print_schema(df)

In [3]:
data

In [4]:
with tf.Graph().as_default() as g:
  
    x = tfs.block(df, "x")
    # The output that adds 3 to x
    z = tf.add(x, 3, name="z")
    # The resulting dataframe
    df2 = tfs.map_blocks(z, df)
    
df2.collect()

In [5]:
# Build a DataFrame of vectors
data = [Row(y=[float(y), float(-y)]) for y in range(10)]
df = sqlContext.createDataFrame(data)
# Because the dataframe contains vectors, we need to analyze it first to find the
# dimensions of the vectors.
df2 = tfs.analyze(df)

# The information gathered by TF can be printed to check the content:
tfs.print_schema(df2)
# TF has inferred that y contains vectors of size 2
# root
#  |-- y: array (nullable = false) DoubleType[?,2]

# Let's use the analyzed dataframe to compute the sum and the elementwise minimum 
# of all the vectors:
# First, let's make a copy of the 'y' column. This will be very cheap in Spark 2.0+
df3 = df2.select(df2.y, df2.y.alias("z"))
with tf.Graph().as_default() as g:
    # The placeholders. Note the special name that end with '_input':
    y_input = tfs.block(df3, 'y', tf_name="y_input")
    z_input = tfs.block(df3, 'z', tf_name="z_input")
    y = tf.reduce_sum(y_input, [0], name='y')
    z = tf.reduce_min(z_input, [0], name='z')
    # The resulting dataframe
    (data_sum, data_min) = tfs.reduce_blocks([y, z], df3)

# The final results are numpy arrays:
print data_sum
# [45.0, -45.0]
print data_min
# [0.0, -9.0]